In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd

#definimos las credenciales

client_id = 'b6841f7c97974c8aacbdf448dae8677f'
client_secret = 'bc15cade85bc41f88d451fbb2c832eff'
redirect_url = 'http://localhost:8888/callback/'

#scope (permisos) para acceder a tus playlist
scope = 'playlist-read-private'

#autenticacion 
sp = spotipy.Spotify(auth_manager = SpotifyOAuth(
    client_id = client_id,
    client_secret = client_secret,
    redirect_uri = redirect_url,
    scope = scope
))

# Obtener tus playlists
playlists = sp.current_user_playlists()
playlists
#mostrar lso nombres de las playlist y sus IDS
for playlist in playlists['items']:
    print(f"Playlist: {playlist['name']}, ID: {playlist['id']}")

Playlist: My recommendation playlist, ID: 3XMMgmodaAPCHcCXBkgr5k
Playlist: My recommendation playlist, ID: 5bQpZDbY0JqW2wvmC5SpB7
Playlist: DJ, ID: 37i9dQZF1EYkqdzj48dyYq
Playlist: duchazo, ID: 4eOY4P1EyXudJoSmZu4uIu
Playlist: myhead, ID: 4Q3DMQ58r0PepiqMMhhGqW


In [2]:
#especifico el ID de mi playist a explorar
playlist_id = '4Q3DMQ58r0PepiqMMhhGqW'

#obtengo los resultados de las canciones de mi playlist
results = sp.playlist_tracks(playlist_id)

In [9]:
# Lista para almacenar todas las canciones
all_tracks = []

# Iterar mientras haya canciones en la respuesta
while results['items']:
    for item in results['items']:
        track = item['track']
        artists_id = track['artists'][0]['id']
        artists_info = sp.artist(artists_id)
        genres = artists_info['genres']
        album = track['album']
        artist_id = track['artists'][0]['id']  # ID del artista principal
        album_image_url = album['images'][0]['url'] if album['images'] else None
        
        # Obtener la información del artista, incluyendo la imagen de perfil
        artist_info = sp.artist(artist_id)
        artist_image_url = artist_info['images'][0]['url'] if artist_info['images'] else None


        all_tracks.append({
            'name': track['name'],
            'artist': track['artists'][0]['name'],
            'album': track['album']['name'],
            'duration_sec': track['duration_ms'] / 1000,
            'popularity': track['popularity'],
            'release_date': track['album']['release_date'],
            'album_image_url': album_image_url,
            'artist_image_url': artist_image_url,
            'genres': ', '.join(genres),
            'track_id':track['id']
        })
    
    # Incrementar el offset para obtener el siguiente bloque de canciones
    offset = len(all_tracks)
    results = sp.playlist_tracks(playlist_id, offset=offset)

# Crear DataFrame con todos los datos de las canciones
playlist_df = pd.DataFrame(all_tracks)


In [10]:
# Separar la columna 'genres' por comas y expandir los resultados en columnas
genres_split = playlist_df['genres'].str.split(', ', expand=True)

# Renombrar las columnas resultantes
genres_split.columns = [f'genre_{i+1}' for i in range(genres_split.shape[1])]

# Convertir todos los valores de la columna 'genres' a cadenas por si acaso hay valores no string
playlist_df['genres'] = playlist_df['genres'].astype(str)

# Contar la cantidad de géneros en cada fila
playlist_df['genre_count'] = playlist_df['genres'].apply(lambda x: len(x.split(', ')))

# Unir el DataFrame original con el DataFrame de géneros separados
playlist_df = pd.concat([playlist_df, genres_split], axis=1)

In [11]:
playlist_df.to_csv('my_playlist.csv')

In [ ]:
playlist_df